In [92]:
import pandas as pd
import re

def tokenize(text):
    if not isinstance(text, str):
        return '[]'
    
    tok = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
    tok = tok.lower().split()
    tok = [t for t in tok if t not in stopwords]
    return tok

def clean(x):
    if not isinstance(x, str):
        return 'no description available'
    else:
        if len(x) < 3:
            return 'no description available'
        else:
            return x
            
def parseYear(accession):
    year = re.search(r'\d+', accession).group()
    return year
    


with open('stopwords.txt') as f:
    stopwords = [w.strip() for w in f.readlines()]
    

primary = pd.read_csv('Data/CSV/05 - Primary Docs.csv')
what = pd.read_csv('Data/CSV/02 – DS What.csv')

primary = primary.drop('Unnamed: 2', axis=1)
primary = primary.drop('Unnamed: 3', axis=1)
primary = primary.drop('Unnamed: 4', axis=1)

what = what.drop('Unnamed: 4', axis=1)
what = what.drop('Description', axis=1)

In [93]:
data = primary.merge(what, on='Accession Number')
data = data.rename(columns={
        'Accession Number': 'accession',
        'Primary documentation': 'primary',
        'Research notes': 'researcher',
        'Publications history': 'literature'
    })

In [94]:
data['c_primary'] = data.primary.apply(tokenize)
data['c_researcher'] = data.researcher.apply(tokenize)
data['c_literature'] = data.literature.apply(tokenize)

data['primary'] = data.primary.apply(clean)
data['researcher'] = data.researcher.apply(clean)
data['literature'] = data.literature.apply(clean)

In [96]:
def f(x):
    if isinstance(x, str):
        y = x.split('.')[0]
        
        if '[' == y[0]:
            y = y[1:]
            
        try:
            y = int(y)
            return y
        except:
            return -1 
    else:
        return -1
   
data['year'] = data.accession.apply(f)

In [98]:
stack = data.c_primary.tolist() + data.c_literature.tolist() + data.c_researcher.tolist()

keywords = []
for l in stack:
    keywords += l
    
print(len(set(keywords)))

74483


In [100]:
from collections import Counter
from pprint import pprint
import json

c = Counter(keywords)
options = []
for k in c.most_common():
    options.append({
        'value': k[0],
        'label': k[0] + ' (' + str([1]) + ')'
    })
    
    
data.to_json('./front/public/data/data.json', orient='records')
    
with open('./front/public/data/options.json', 'w') as f:
    #with open('options.json', 'w') as f:
    json.dump(options, f, ensure_ascii=False, indent=4)